In [ ]:

from fastembed import SparseTextEmbedding
from qdrant_client import QdrantClient, models
import os
import json
from langchain_openai import AzureOpenAIEmbeddings
from qdrant_client import QdrantClient
client = QdrantClient("http://localhost:6333", timeout=300)

In [ ]:

MODEL_NAME = "prithivida/Splade_PP_en_v1"
#MODEL_NAME= "Qdrant/bm42-all-minilm-l6-v2-attentions"
#MODEL_NAME = "Qdrant/bm25"
SPARSE_TYPE = "bm42"
DATASET = "quora"
COLLECTION_NAME = "quora_collection"

In [ ]:

def load_queries(dataset):
    queries = {}

    with open(dataset, "r") as file:
        for line in file:
            row = json.loads(line)
            queries[row["_id"]] = row["text"]
    return queries

In [ ]:

DATASET_q = f"/datasets/{DATASET}/queries.jsonl"
queries = load_queries(DATASET_q)

In [ ]:
# this function encodes a query into a sparse vector using a specified model (bm25/splade/BM42)
def encode_sparse_query(query, model_name= MODEL_NAME):  
    model = SparseTextEmbedding(model_name)
    embedding = list(model.query_embed(query))[0]
    sparse_vector = models.SparseVector(values=embedding.values.tolist(), indices=embedding.indices.tolist())
    return sparse_vector

In [ ]:
def sparse_search(sparse_query):
    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=sparse_query,
        using=SPARSE_TYPE,
        with_payload=True,
        #with_vectors=["bm42"],
        limit=10
    )
    return results

In [ ]:
with open("./quora_bm42_sparse.jsonl", "w") as dense_out:
    for idx,text in queries.items():
        sparse_vector = encode_sparse_query(text)
        search_result = sparse_search(sparse_vector)

        sparse_output = {"query_id": idx,"results":
                        [{"doc_id": point.id,"text": point.payload, "score": point.score}
                         for point in search_result.points]}
        dense_out.write(json.dumps(sparse_output) + "\n")